In [1]:
import pandas as pd
import requests
import time
import threading
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException

In [2]:
pip install beautifulsoup4 selenium


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install selenium --upgrade


Note: you may need to restart the kernel to use updated packages.


## Collecting Data Processing

To develop the machine learning model and obtain the data for the training set, information from the real estate website was used. This was because certain characteristics of the properties were essential. However, since this information wasn't already tabulated, web scraping techniques were employed, such as the Selenium library. Below are the details on how it was developed.


## First Step

The next script is designed to gather property links from a real estate website using automated web browsing tools, specifically a library called Selenium. This lets the program act like a person using a web browser, scrolling through pages, waiting for new content to appear, and then grabbing the links to properties.

The program is smart enough to handle lots of pages efficiently. Instead of going through each page one by one, it can work on several pages at the same time. This is done using something called multithreading, which is like having multiple hands all working together. And if the program runs into any problems, like a page not loading properly, it will try again a few times before moving on.

Once all the property links are collected, the program shows them so you can see the results. This makes it easy to check the work and use those links for other things later on.

In [ ]:

MAX_RETRIES = 3

def extract_links(driver):
    links = []

    max_scrolls = 20
    scrolls = 0

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
    time.sleep(20)

    while True:
        try:
            house_links_elements = driver.find_elements(By.CSS_SELECTOR, "a.ember-view.listing-tile-info")

            for link in house_links_elements:
                link_href = link.get_attribute('href')
                if link_href not in links:
                    links.append(link_href)

            if len(links) >= 10:
                break

            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(3)

            scrolls += 1
            if scrolls >= max_scrolls:
                break

        except WebDriverException:
            return []

    return links

def extract_links_from_pages(driver, start_page, end_page):
    all_links = []
    base_url = 'https://www.realestate.co.nz/residential/rental/auckland/auckland-city?by=oldest'
    driver.execute_script("document.body.style.zoom = '0.5'")
    for i in range(start_page, end_page + 1):
        retries = 0
        success = False
        while retries < MAX_RETRIES and not success:
            try:
                if i > 1:
                    url = f"{base_url}&page={i}"
                else:
                    url = base_url
                driver.get(url)
                page_links = extract_links(driver)
                all_links.extend(page_links)
                print(f"Extracted {len(page_links)} links from page {i}")
                success = True
            except WebDriverException:
                retries += 1
                print(f"Error loading page {i}. Retrying ({retries}/{MAX_RETRIES})...")
                if retries == MAX_RETRIES:
                    print(f"Failed to load page {i} after {MAX_RETRIES} retries. Moving to next page.")

    return all_links

def start_extraction_thread(start_page, end_page):
    options = webdriver.ChromeOptions()
    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_setting_values.notifications": 2,
        "profile.default_content_setting_values.popups": 2
    }
    options.add_experimental_option("prefs", prefs)
    options.add_argument("--disable-popup-blocking")

    while start_page <= end_page:
        try:
            driver = webdriver.Chrome(options=options)
            driver.set_script_timeout(300)
            links = extract_links_from_pages(driver, start_page, end_page)
            for link in links:
                print(link)
            driver.quit()
            break  # Exit the loop if extraction was successful
        except WebDriverException:
            print("Driver crashed. Restarting...")
            start_page = end_page  # Adjust the start_page if needed

if __name__ == "__main__":
    t1 = threading.Thread(target=start_extraction_thread, args=(1, 20))
    t2 = threading.Thread(target=start_extraction_thread, args=(21, 40))

    t1.start()
    t2.start()

    t1.join()
    t2.join()



In [4]:
url = 'https://raw.githubusercontent.com/robertoaltran/Population/main/links.txt'
response = requests.get(url)
sites = response.text.splitlines()
sites = [site.strip() for site in sites]


In [5]:
quantity_elements = len(sites)
print("Elements:", quantity_elements)

Elements: 405


In [6]:
sites.sort()

In [71]:
print(sites[:5])

['https://www.realestate.co.nz/3745538/residential/rent/20-9a-esplanade-road-mount-eden', 'https://www.realestate.co.nz/42187622/residential/rent/46-montrose-street-point-chevalier', 'https://www.realestate.co.nz/42193564/residential/rent/1-351-mount-eden-road-mount-eden', 'https://www.realestate.co.nz/42195889/residential/rent/64-school-road-kingsland', 'https://www.realestate.co.nz/42202198/residential/rent/2e-25-rutland-street-auckland-central']


In [9]:
sites = [url for url in sites if not url.endswith('rent')]
sites.remove('Extracted 10 links from page 20')


In [10]:
siteurls = sites[:]

In [11]:
quantity_elements = len(sites)
print("Elements:", quantity_elements)

Elements: 389


After collecting the links and evaluating their quality, we removed any potentially unreliable links. We ended up with 389 valid elements to be visited.

-----------------------------------------------------------

## 2 step

The following algorithm is a web scraper that utilizes Selenium, an automation tool, to navigate and fetch data from specific websites. Its primary purpose is to extract information about properties, such as addresses and prices, from provided URLs and store this data neatly into a structured format.

To maximize efficiency and speed, the script breaks down the list of websites into chunks and processes them simultaneously, using a technique called multithreading. This is akin to deploying multiple virtual browsers all working together. If the program encounters issues while scraping a site, it's designed to retry a couple of times before logging an error and proceeding to the next site.

After all the property information is compiled, the script saves everything into a CSV file for easy viewing and further analysis. This ensures that even large amounts of data are preserved and presented in a user-friendly manner.

In [ ]:
import threading
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd

MAX_RETRIES = 2

def initialize_driver():
    options = webdriver.ChromeOptions()
    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_setting_values.notifications": 2,
        "profile.default_content_setting_values.popups": 2
    }
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    return driver

def extract_info_chunk(start_idx, end_idx, driver, info_chunk_list):
    for i in range(start_idx, end_idx):
        url = sites[i]
        info_dict = extract_info((url, driver))
        info_chunk_list.append(info_dict)
        print(info_dict)  

def extract_info(args):
    url, driver = args
    info_dict = {}
    
    retries = 0
    while retries < MAX_RETRIES:
        try:
            driver.get(url)
            print(f"Scraping: {url}")

            address_element = driver.find_element(By.CSS_SELECTOR, "h1[data-test='listing-title']")
            address = address_element.text.strip()
            info_dict["Address"] = address

            price_element = driver.find_element(By.CSS_SELECTOR, "h3.text-xl.font-semibold")
            price = price_element.text.strip()
            info_dict["Price"] = price

            elements = driver.find_elements(By.CSS_SELECTOR, "span.leading-\\[-1\\]")
            max_elements_to_extract = 8
            for i, element in enumerate(elements, start=1):
                if i > max_elements_to_extract:
                    break
                text = element.text.strip()
                info_dict[f"Info_{i}"] = text

            if len(info_dict) >= 6:
                break

        except (NoSuchElementException, TimeoutException) as e:
            retries += 1
            if retries == MAX_RETRIES:
                info_dict["Error"] = str(e)
                break

    return info_dict

if __name__ == "__main__":
   
    drivers = [initialize_driver() for _ in range(3)]
   
    info_list1 = []
    info_list2 = []
    info_list3 = []

    third_length = len(siteurls) // 3

    t1 = threading.Thread(target=extract_info_chunk, args=(0, third_length, drivers[0], info_list1))
    t2 = threading.Thread(target=extract_info_chunk, args=(third_length, 2 * third_length, drivers[1], info_list2))
    t3 = threading.Thread(target=extract_info_chunk, args=(2 * third_length, len(sites), drivers[2], info_list3))

    # iniciating threads
    t1.start()
    t2.start()
    t3.start()

    t1.join()
    t2.join()
    t3.join()

    # joinning results
    info_list = info_list1 + info_list2 + info_list3

    # closing drivers
    for driver in drivers:
        driver.quit()

    df = pd.DataFrame(info_list)
    df.to_csv('scraped_data.csv', index=False)
    print("\nScraping results:")
    print(df)



"The script ran smoothly; however, it took about 15 hours to process all 389 links."





In [12]:
import re

url = "https://raw.githubusercontent.com/robertoaltran/Population/main/scraped_data.csv"
df = pd.read_csv(url)
df = df.dropna(subset=['Address'])
temp_filename = 'temp.csv'
df.to_csv(temp_filename, index=False)
with open(temp_filename, 'r') as file:
    content = file.read()
corrected_content = re.sub(r'\n(?=[^"])', ' ', content)
corrected_temp_filename = 'corrected_temp.csv'
with open(corrected_temp_filename, 'w') as file:
    file.write(corrected_content)
df_corrected = pd.read_csv(corrected_temp_filename)

In [13]:
df = df_corrected

In [14]:
def is_numeric(value):
    #Check if the value is numeric.
    if pd.isna(value):
        return False
    try:
        int(value)
        return True
    except ValueError:
        return False

In [15]:
column_rename = {
    'Info_1': 'House Type',
    'Info_2': 'Bedroom',
    'Info_3': 'Bathroom',
    'Info_4': 'Garage',
    'Info_5': 'Other Feature 1',
    'Info_6': 'Other Feature 2',
}

df.rename(columns=column_rename, inplace=True)
df.drop(['Info_7', 'Info_8', 'Error'], axis=1, inplace=True)

In [16]:
# Convert 'Price' column to numerical by removing unwanted strings
df['Price'] = df['Price'].astype(str)

df['Price'] = df['Price'].str.replace(r'\$', '', regex=True)
df['Price'] = df['Price'].str.replace(r' per week', '', regex=True)
df['Price'] = df['Price'].str.replace(r',', '', regex=True)
#df['Price'] = pd.to_numeric(df['Price'], errors='coerce')  # Convert to numeric (float)

# Handle numeric values in 'Bedroom', 'Bathroom' and 'Garage'
df['Bedroom'] = df['Bedroom'].apply(lambda x: int(x) if is_numeric(x) else None)
df['Bathroom'] = df['Bathroom'].apply(lambda x: int(x) if is_numeric(x) else None)
mask_non_numeric_garage = ~df['Garage'].apply(is_numeric)
df.loc[mask_non_numeric_garage, ['Garage', 'Other Feature 1', 'Other Feature 2']] = df.loc[mask_non_numeric_garage, ['Other Feature 1', 'Garage', 'Other Feature 2']].values

# Convert the columns to Int64 type
for col in ['Bedroom', 'Bathroom', 'Garage']:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype(pd.Int64Dtype())

# Handle cases where 'Other Feature 2' is numeric but 'Other Feature 1' isn't
mask = df['Other Feature 2'].apply(is_numeric) & ~df['Other Feature 1'].apply(is_numeric)
df.loc[mask, ['Other Feature 1', 'Other Feature 2']] = df.loc[mask, ['Other Feature 2', 'Other Feature 1']].values

# Add up numeric 'Garage' and 'Other Feature 1' values
mask = df['Garage'].apply(is_numeric) & df['Other Feature 1'].apply(is_numeric)
df.loc[mask, 'Garage'] += df.loc[mask, 'Other Feature 1'].astype(int)
df.loc[mask, 'Other Feature 1'] = df.loc[mask, 'Other Feature 2']
df.loc[mask, 'Other Feature 2'] = None

# Handle 'Furniture'
df['Furniture'] = 0
furnished_mask = df['Other Feature 1'].str.contains('Furnished', na=False) | df['Other Feature 2'].str.contains('Furnished', na=False)
df.loc[furnished_mask, 'Furniture'] = 1
df.loc[furnished_mask, ['Other Feature 1', 'Other Feature 2']] = None

non_furnished_mask = df['Other Feature 1'].str.contains('Non furnished', na=False) | df['Other Feature 2'].str.contains('Non furnished', na=False)
df.loc[non_furnished_mask, ['Other Feature 1', 'Other Feature 2']] = None

# Impute missing values in 'House Type'
df['House Type'].fillna('Unknown', inplace=True)

# One-hot encoding for 'House Type'
df = pd.get_dummies(df, columns=['House Type'], prefix='Type')

# Ensure all categories are present and if not, add them
categories = ["House", "Apartment", "Unit", "Townhouse", "Unknown"]
for cat in categories:
    col_name = 'Type_' + cat
    if col_name not in df.columns:
        df[col_name] = 0


In [17]:
df.head()

,Address,Price,Bedroom,Bathroom,Garage,Other Feature 1,Other Feature 2,Furniture,Type_Apartment,Type_Carpark,Type_House,Type_Studio,Type_Townhouse,Type_Unit,Type_Unknown
0,"20/9A Esplanade Road, Mount Eden, Auckland City",360,1,1,1,NaN,NaN,0,0,0,0,0,0,1,0
1,"46 Montrose Street, Point Chevalier, Auckland ...",650,2,1,2,NaN,NaN,0,0,0,1,0,0,0,0
2,"1/351 Mount Eden Road, Mount Eden, Auckland City",530,2,1,<NA>,NaN,NaN,0,0,0,1,0,0,0,0
3,"64 School Road, Kingsland, Auckland City",750,3,1,2,NaN,NaN,0,0,0,1,0,0,0,0
4,"2E/25 Rutland Street, Auckland Central, Auckla...",370,1,1,<NA>,None,None,0,1,0,0,0,0,0,0


In [18]:
df['Suburb'] = df['Address'].str.split(',').str[-2].str.strip()


In [19]:
df.head()

,Address,Price,Bedroom,Bathroom,Garage,Other Feature 1,Other Feature 2,Furniture,Type_Apartment,Type_Carpark,Type_House,Type_Studio,Type_Townhouse,Type_Unit,Type_Unknown,Suburb
0,"20/9A Esplanade Road, Mount Eden, Auckland City",360,1,1,1,NaN,NaN,0,0,0,0,0,0,1,0,Mount Eden
1,"46 Montrose Street, Point Chevalier, Auckland ...",650,2,1,2,NaN,NaN,0,0,0,1,0,0,0,0,Point Chevalier
2,"1/351 Mount Eden Road, Mount Eden, Auckland City",530,2,1,<NA>,NaN,NaN,0,0,0,1,0,0,0,0,Mount Eden
3,"64 School Road, Kingsland, Auckland City",750,3,1,2,NaN,NaN,0,0,0,1,0,0,0,0,Kingsland
4,"2E/25 Rutland Street, Auckland Central, Auckla...",370,1,1,<NA>,None,None,0,1,0,0,0,0,0,0,Auckland Central


In [20]:
unique_values = df['Suburb'].unique()
string_values = [value for value in unique_values if isinstance(value, str)]
sorted_unique_values = sorted(string_values)

for value in sorted_unique_values:
    print(value)

Auckland Central
Avondale
Blockhouse Bay
Eden Terrace
Ellerslie
Epsom
Freemans Bay
Glen Innes
Glendowie
Grafton
Greenlane
Grey Lynn
Herne Bay
Hillsborough
Kingsland
Kohimarama
Meadowbank
Mission Bay
Morningside
Mount Albert
Mount Eden
Mount Roskill
Mount Wellington
New Windsor
Newmarket
One Tree Hill
Onehunga
Otahuhu
Parnell
Point Chevalier
Point England
Ponsonby
Remuera
Royal Oak
Saint Heliers
Saint Johns
Saint Marys Bay
Sandringham
St Heliers
Stonefields
Three Kings
Waiotaiki Bay
Waterview
Westmere


In [21]:
df.to_csv('RealstateRent.csv', index=False)


In [22]:
dfinal = df

This program is made to fix and organize data about properties from an online file.

First, the program gets data from a website link and removes any entries without an address. It saves this data to a file, checks for any mistakes like extra lines, and then corrects them. After making these fixes, it reads the data back and gets ready for more detailed changes.

The main job of the program is to make the property data easy to understand. It gives new names to some columns to make them clearer. It removes unnecessary information and changes the price details to simple numbers by getting rid of symbols and words. The program also checks details like the number of rooms and parking spaces, making sure they are in the right place and shown as numbers. It also looks for houses that come with or without furniture, marking them correctly. At the end, it changes the property type (like "House" or "Apartment") into a format that a computer can easily understand. If any type is missing, the program adds it. Now, the cleaned-up data is ready for more use, like studying or making predictions.

----------------------------------------------------------------

## 3 step

Gathering good data is very important for study. For this project, data was looked into different areas using the police website and nz.statistics. And it was mainly used the 2018 census for information. Even though it was from 2018, it was updated in 2020, so it's still recent. 

In [23]:
pd.set_option('display.max_columns', None)


In [24]:
!pip install pandas openpyxl

In [25]:
url = 'https://raw.githubusercontent.com/robertoaltran/Population/main/2018-SA1-dataset-household-AucklandRegion.csv'
data2 = pd.read_csv(url)

In [26]:
data2 = data2.iloc[:, [0, 1, 4]]
headers = data2.iloc[0]
data2 = data2[1:]
data2.columns = headers
data2.head()

,Area_Code,Area_Description,2018.0
1,110200,Okahukura Peninsula,522.0
2,110300,Inlet Kaipara Harbour South,0.0
3,110400,Cape Rodney,1275.0
4,110500,Wellsford,654.0
5,110600,Oceanic Auckland Region East,6.0


### Total households:

This metric shows the number of homes currently being lived in. It can help gauge housing demand in an area.

-----------------

In [27]:
url = 'https://raw.githubusercontent.com/robertoaltran/Population/main/2018-SA1-dataset-individual-part-1-AucklandRegion_updated_28-7-20.csv'
data3 = pd.read_csv(url)

In [28]:
data3 = data3.iloc[:, [0, 1, 4] + list(range(61, 78)) + [80] + list(range(91, 94)) + list(range(346, 353))]

In [29]:
headers = data3.iloc[0]
data3 = data3[1:]
data3.columns = headers
data3.head()

,Area_Code,Area_Description,2018 Census,0-4 years,5-9 years,10-14 years,15-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,Median age(17),Under 15 years,15-29 years,30-64 years,Less than one year,1 year,2 years,3 years,4 years,5-9 years,10-19 years
1,110200,Okahukura Peninsula,1491,99,96,111,84,75,72,78,93,87,99,105,162,105,84,51,48,21,41.7,306,234,732,6,3,3,9,3,33,60
2,110300,Inlet Kaipara Harbour South,0,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C
3,110400,Cape Rodney,3525,216,234,237,168,153,168,177,201,204,240,264,312,291,258,198,123,51,45,687,489,1683,27,21,21,18,21,66,138
4,110500,Wellsford,1929,168,150,129,138,105,138,150,123,105,93,114,108,84,99,87,57,42,34.4,447,384,777,12,18,6,18,6,78,78
5,110600,Oceanic Auckland Region East,9,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,62,0,0,6,C,C,C,C,C,C,C



**Census usually resident population count**: 
  The total number of people who live in an area most of the time.

**Age in five year groups**: 
 Grouping people by age, like 0-5 years, 6-10 years, and so on.

**Age in broad groups**: 
 Grouping people by age in larger groups, like children, adults, and seniors.

**Years at usual residence**: 
 How many years a person has lived in their current home.

updated_28-7-20

----------------

In [30]:
url = 'https://raw.githubusercontent.com/robertoaltran/Population/main/2018-SA1-dataset-individual-part-2-AucklandRegion.csv'
data4 = pd.read_csv(url)

In [31]:
data4 = data4.iloc[:, [0, 1] + list(range(174, 180)) + list(range(215, 226)) + list(range(264, 270))+ [274]]

In [32]:
headers = data4.iloc[0]
data4 = data4[1:]
data4.columns = headers

data4.head()

,Area_Code,Area_Description,No children,One child,Two children,Three children,Four children,Five children,No qualification,Level 1 certificate,Level 2 certificate,Level 3 certificate,Level 4 certificate,Level 5 diploma,Level 6 diploma,Bachelor degree and Level 7 qualification,Post-graduate and honours degrees,Masters degree,Doctorate degree,"$5,000 or less","$5,001 – $10,000","$10,001 – $20,000","$20,001 – $30,000","$30,001 – $50,000","$50,001 – $70,000",Median personal income($)
1,110200,Okahukura Peninsula,108,93,141,90,48,21,225,198,156,135,120,54,45,93,30,15,3,132,57,198,168,288,177,32600
2,110300,Inlet Kaipara Harbour South,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C
3,110400,Cape Rodney,315,156,378,246,120,33,495,357,321,249,315,129,150,333,102,81,39,312,123,504,408,642,393,32100
4,110500,Wellsford,150,87,159,144,75,33,381,219,186,141,132,57,48,90,30,15,0,192,72,321,270,303,195,25400
5,110600,Oceanic Auckland Region East,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,12600


### Feature Relevance for House Rental Price Model:

When predicting house rental prices in New Zealand, it's essential to determine the relevance of potential features. Below is a brief overview:


**Number of Children Born**: Can influence demand for property type and size.

**Education Metrics**: Locations near educational institutions or with a high student population might have specific rental demands.

**Income Metrics**: Areas with higher incomes might exhibit higher rental prices. The source of income can offer additional insights.

**Travel Means to Education**: Indicates transport infrastructure and proximity to educational establishments.

-------------------------------

In [33]:
url = 'https://raw.githubusercontent.com/robertoaltran/Population/main/2018-SA1-dataset-individual-part-3a-AucklandRegion.csv'
data5 = pd.read_csv(url)

In [34]:
data5 = data5.iloc[:, [0, 1] + list(range(16, 21)) + list(range(66, 73)) + list(range(156, 172))]

In [35]:
headers = data5.iloc[0]
data5 = data5[1:]
data5.columns = headers
data5.head()

,Area_Code,Area_Description,Employed Full time,Employed Part time,Unemployed,Not in the Labour Force,Total stated,Managers,Professionals,Technicians and Trades Workers,Community and Personal Service Workers,Clerical and Administrative Workers,Sales Workers,Machinery Operators and Drivers,Manufacturing,Electricity Gas Water and Waste Services,Construction,Wholesale Trade,Retail Trade,Accommodation and Food Services,Transport Postal and Warehousing,Information Media and Telecommunications,Financial and Insurance Services,Rental Hiring and Real Estate Services,Professional Scientific and Technical Services,Administrative and Support Services,Public Administration and Safety,Education and Training,Health Care and Social Assistance,Arts and Recreation Services
1,110200,Okahukura Peninsula,639,198,39,306,1182,204,93,117,75,66,66,81,72,3,114,42,57,39,42,6,12,15,42,39,24,48,54,9
2,110300,Inlet Kaipara Harbour South,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C
3,110400,Cape Rodney,1329,558,48,903,2838,507,309,246,147,186,147,93,165,6,264,90,126,111,60,27,21,51,156,72,24,141,135,57
4,110500,Wellsford,681,213,66,525,1485,156,96,144,78,87,117,81,90,3,153,45,114,78,54,3,15,6,63,33,15,54,54,12
5,110600,Oceanic Auckland Region East,C,C,C,6,9,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C



**Work and Labour Force Status**:
- This shows how many people in an area have jobs. Places with more people working might have higher rent prices because more people need homes.

**Status in Employment**:
- This tells us what kind of jobs people have, like full-time or part-time. Areas with many full-time workers might have higher rents because they have a steady income.

**Occupation by Residence & Workplace Address**:
- This information reveals the types of jobs people do and where they work. Areas with lots of high-paying jobs, like doctors or lawyers, might have higher rents. Also, if people live close to their work, they might be willing to pay more for rent.

**Industry by Residence & Workplace Address**:
- This tells us about the businesses and industries where people work. For instance, areas close to big business hubs might have higher rents. On the other hand, places with seasonal jobs might have different rent patterns.

-------------------------

In [36]:
url = 'https://raw.githubusercontent.com/robertoaltran/Population/main/2018-SA1-dataset-individual-part-3b-AucklandRegion_updated_16-7-20.csv'
data6 = pd.read_csv(url, header = 1)

In [37]:
data6 = data6.iloc[:, [0, 1, 32 , 38, 41, 44, 50, 53, 56]]

In [38]:
data6.drop(0, inplace=True)
data6.reset_index(drop=True, inplace=True)
data6.rename(columns={'Unnamed: 0': 'Area_Code', 'Unnamed: 1': 'Area_Description'}, inplace=True)

In [39]:
data6.head()

,Area_Code,Area_Description,Work at home,"Drive a private car, truck or van","Drive a company car, truck or van","Passenger in a car, truck, van or company bus",Train,Bicycle,Walk or jog
0,110200,Okahukura Peninsula,168,315,78,12,0,0,18
1,110300,Inlet Kaipara Harbour South,C,C,C,C,C,C,C
2,110400,Cape Rodney,375,564,129,39,3,6,33
3,110500,Wellsford,57,315,108,39,0,3,81
4,110600,Oceanic Auckland Region East,C,C,C,C,C,C,C


**Main Means of Travel to Work by Residence & Workplace Address**:
- This shows how people get to their jobs, like by car, bus, or walking. Areas with good public transport might have higher rents because it's easier for people to get around. On the other hand, places where most people drive might need homes with parking spaces.

------------------------------------------

In [40]:
url = 'https://raw.githubusercontent.com/robertoaltran/Population/main/ANZSOC_Full%20Data_data-auckland.csv'  
data8 = pd.read_csv(url)
data8.head()

,ANZSOC Division,Year Month,Victimisations,Area Unit,Number of Records,Territorial Authority
0,"Abduction, Harassment and Other Related Offenc...",Dec-22,1,Mangere South.,1,Auckland.
1,"Abduction, Harassment and Other Related Offenc...",Oct-22,1,Royal Oak.,1,Auckland.
2,"Abduction, Harassment and Other Related Offenc...",Dec-22,1,Mascot.,1,Auckland.
3,"Abduction, Harassment and Other Related Offenc...",Dec-22,1,Balmoral.,1,Auckland.
4,"Abduction, Harassment and Other Related Offenc...",Oct-22,1,Drury.,1,Auckland.


In [41]:
data8['Area Unit'] = data8['Area Unit'].str.replace(r'\.$', '', regex=True)

## Auckland - Crime Categories and Incidents:

**Acts Intended to Cause Injury**
This category involves deliberate acts where the intent was to inflict harm or injury to another individual.

**Sexual Assault and Related Offences**
These are offenses that involve any unwanted sexual activity or behavior without the explicit consent of the victim.

**Abduction, Harassment and Other Related Offences Against Personal Liberty**
This category encompasses crimes where a person's freedom or liberty is violated, including kidnapping and harassment.

**Robbery, Extortion and Related Offences**
Such crimes involve forcibly taking something of value from another person, often coupled with threats or violence.

**Unlawful Entry With Intent/Burglary, Break and Enter**
These are offenses where individuals unlawfully enter property, usually with the intent to commit theft or another crime.

**Theft and Related Offences**
This category covers various forms of theft, from petty thefts to more significant robberies, excluding break-ins.


### Source of Data:
The data is derived from the "Victimisation Time and Place" dataset, which offers detailed insights into when and where crimes occurred across Auckland.

------------------------------

## Data Cleaning

With the gathered data on the properties and their respective neighborhoods, a crucial next step is data cleaning and preprocessing. Ensuring that data is accurate and in a usable format is important to the success of any machine learning model. We have done on a comprehensive process to structure the data effectively. This involves combining information from different sources, making it cohesive and consistent. Cleaning techniques such as using pivot tables played and corrected instances of duplicate names and undertook reformatting measures. These steps ensure that the data is not only accurate but optimized for the model, making it better for efficient analysis.

In [42]:
data_crimes_2023 = data8[data8['Year Month'].str.contains('23')]
grouped_crimes = data_crimes_2023.groupby(['Area Unit', 'ANZSOC Division'])['Victimisations'].sum().reset_index()
pivot_crimes = grouped_crimes.pivot_table(index='Area Unit', columns='ANZSOC Division', values='Victimisations', fill_value=0).reset_index()

In [43]:
pivot_crimes.head()

ANZSOC Division,Area Unit,"Abduction, Harassment and Other Related Offences Against a Person",Acts Intended to Cause Injury,"Robbery, Extortion and Related Offences",Sexual Assault and Related Offences,Theft and Related Offences,"Unlawful Entry With Intent/Burglary, Break and Enter"
0,-29,0,2,0,0,7,1
1,999999,1,66,16,28,204,28
2,Abbotts Park,0,8,4,1,197,50
3,Akarana,0,20,2,1,96,73
4,Albany,1,80,15,4,734,63


In [44]:
unique_value = pivot_crimes['Area Unit'].unique()
string_values = [value for value in unique_value if isinstance(value, str)]
sorted_unique_value = sorted(string_values)


In [45]:
pivot_crimes['Area Unit'] = pivot_crimes['Area Unit'].str.replace(r' (North|South|East|West|Central)\b', '', regex=True)
pivot_crimes['Area Unit'] = pivot_crimes['Area Unit'].str.replace(r' \(Auckland\)', '', regex=True)

In [46]:
result = pivot_crimes.groupby('Area Unit').sum().reset_index()

In [47]:
result['Area Unit'] = result['Area Unit'].replace(['Auckland City-Marinas', 'Auckland Harbourside'], 'Auckland')

In [48]:
dataframes = [data2, data3, data4, data5, data6]
names = ['data2', 'data3', 'data4', 'data5', 'data6']

for df, name in zip(dataframes, names):
    if 'Area_Description' not in df.columns:
        print(f"{name} doesn't have 'Area_Description' column.")

In [49]:
merged_df = data2.merge(data3, on=['Area_Code', 'Area_Description'], how='outer')
merged_df = merged_df.merge(data4, on=['Area_Code', 'Area_Description'], how='outer')
merged_df = merged_df.merge(data5, on=['Area_Code', 'Area_Description'], how='outer')
merged_df = merged_df.merge(data6, on=['Area_Code', 'Area_Description'], how='outer')

In [50]:
merged_final = merged_df

In [51]:
unique_values = merged_df['Area_Description'].unique()
string_values = [value for value in unique_values if isinstance(value, str)]
sorted_unique_values = sorted(string_values)


In [52]:
merged_final['Area_Description'] = merged_final['Area_Description'].str.replace(r' (North|South|East|West|Central)\b', '', regex=True)
merged_final['Area_Description'] = merged_final['Area_Description'].str.replace(r' \(Auckland\)', '', regex=True)

In [53]:
object_cols = merged_final.select_dtypes(include=['object']).columns
for col in object_cols:
    try:
        _ = pd.to_numeric(merged_final[col], errors='coerce')
    except Exception as e:
        print(f"Error in column {col}: {e}")


Error in column 5-9 years: arg must be a list, tuple, 1-d array, or Series
Error in column 5-9 years: arg must be a list, tuple, 1-d array, or Series


In [54]:
cols = merged_final.columns.tolist()
index_of_duplicate = cols.index("5-9 years")
cols[index_of_duplicate] = "5-9 year"
merged_final.columns = cols

In [55]:
import numpy as np
object_cols_list = object_cols.tolist()

if 'Area_Description' in object_cols_list:
    object_cols_list.remove('Area_Description')

for col in object_cols_list:
    merged_final[col] = merged_final[col].replace('C', np.nan)
    merged_final[col] = pd.to_numeric(merged_final[col], errors='coerce')

In [56]:
merged_final = merged_final.sort_values(by='Area_Description')

merged_final.head()

,Area_Code,Area_Description,2018.0,2018 Census,0-4 years,5-9 year,10-14 years,15-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,Median age(17),Under 15 years,15-29 years,30-64 years,Less than one year,1 year,2 years,3 years,4 years,5-9 years,10-19 years,No children,One child,Two children,Three children,Four children,Five children,No qualification,Level 1 certificate,Level 2 certificate,Level 3 certificate,Level 4 certificate,Level 5 diploma,Level 6 diploma,Bachelor degree and Level 7 qualification,Post-graduate and honours degrees,Masters degree,Doctorate degree,"$5,000 or less","$5,001 – $10,000","$10,001 – $20,000","$20,001 – $30,000","$30,001 – $50,000","$50,001 – $70,000",Median personal income($),Employed Full time,Employed Part time,Unemployed,Not in the Labour Force,Total stated,Managers,Professionals,Technicians and Trades Workers,Community and Personal Service Workers,Clerical and Administrative Workers,Sales Workers,Machinery Operators and Drivers,Manufacturing,Electricity Gas Water and Waste Services,Construction,Wholesale Trade,Retail Trade,Accommodation and Food Services,Transport Postal and Warehousing,Information Media and Telecommunications,Financial and Insurance Services,Rental Hiring and Real Estate Services,Professional Scientific and Technical Services,Administrative and Support Services,Public Administration and Safety,Education and Training,Health Care and Social Assistance,Arts and Recreation Services,Work at home,"Drive a private car, truck or van","Drive a company car, truck or van","Passenger in a car, truck, van or company bus",Train,Bicycle,Walk or jog
170,127200.0,Akoranga,405.0,1167.0,42.0,39,27.0,153.0,105.0,84.0,57.0,45.0,33.0,48.0,39.0,36.0,39.0,33.0,48.0,69.0,87.0,43.9,108.0,342.0,291.0,21.0,18.0,18.0,18.0,12.0,48.0,126.0,237.0,66.0,111.0,87.0,36.0,24.0,147.0,87.0,78.0,195.0,51.0,36.0,75.0,132.0,45.0,36.0,6.0,168.0,78.0,201.0,267.0,192.0,81.0,22100.0,285.0,129.0,51.0,591.0,1056.0,48.0,96.0,45.0,72.0,42.0,60.0,9.0,21.0,3.0,33.0,18.0,45.0,51.0,15.0,9.0,12.0,3.0,51.0,24.0,15.0,27.0,48.0,18.0,12.0,135.0,21.0,9.0,0.0,3.0,36.0
71,117300.0,Albany,162.0,525.0,18.0,12,6.0,93.0,96.0,54.0,57.0,30.0,36.0,30.0,18.0,18.0,21.0,15.0,6.0,3.0,3.0,28.4,39.0,246.0,213.0,45.0,24.0,30.0,18.0,9.0,51.0,72.0,156.0,24.0,30.0,9.0,3.0,3.0,24.0,21.0,27.0,126.0,27.0,9.0,12.0,90.0,21.0,27.0,3.0,144.0,33.0,90.0,39.0,78.0,54.0,17400.0,201.0,90.0,30.0,171.0,489.0,45.0,84.0,21.0,36.0,30.0,42.0,12.0,21.0,0.0,21.0,18.0,48.0,15.0,9.0,9.0,9.0,0.0,36.0,12.0,9.0,36.0,18.0,3.0,6.0,42.0,3.0,6.0,0.0,0.0,3.0
72,117400.0,Albany,882.0,2811.0,126.0,189,171.0,183.0,234.0,156.0,162.0,204.0,171.0,180.0,159.0,144.0,93.0,90.0,138.0,144.0,117.0,39.6,483.0,570.0,1104.0,141.0,159.0,108.0,72.0,60.0,204.0,447.0,318.0,165.0,384.0,177.0,54.0,30.0,306.0,204.0,159.0,216.0,123.0,93.0,117.0,387.0,126.0,135.0,21.0,465.0,117.0,345.0,333.0,420.0,249.0,26100.0,852.0,324.0,72.0,1080.0,2325.0,240.0,300.0,123.0,87.0,141.0,168.0,51.0,93.0,6.0,105.0,102.0,153.0,84.0,30.0,36.0,42.0,39.0,159.0,51.0,36.0,90.0,87.0,15.0,78.0,534.0,138.0,36.0,3.0,6.0,33.0
79,118100.0,Albany,1143.0,3405.0,171.0,186,198.0,222.0,306.0,231.0,243.0,237.0,219.0,264.0,234.0,189.0,195.0,168.0,114.0,81.0,51.0,38.0,555.0,756.0,1578.0,132.0,150.0,117.0,66.0,69.0,312.0,645.0,465.0,234.0,438.0,174.0,42.0,15.0,258.0,201.0,195.0,324.0,171.0,129.0,123.0,537.0,180.0,192.0,30.0,534.0,153.0,381.0,354.0,531.0,375.0,30300.0,1317.0,399.0,99.0,1035.0,2853.0,354.0,459.0,165.0,132.0,231.0,213.0,63.0,120.0,12.0,129.0,153.0,183.0,126.0,63.0,36.0,93.0,39.0,249.0,75.0,66.0,150.0,135.0,36.0,84.0,945.0,240.0,63.0,0.0,9.0,57.0
65,116700.0,Albany Heights,1053.0,3153.0,210.0,192,165.0,189.0,375.0,300.0,312.0,288.0,195.0,186.0,201.0,186.0,153.0,93.0,45.0,33.0,18.0,32.4,567.0,864.0,1521.0,132.0,177.0,114.0,120.0,102.0,393.0,576.0,486.0,234.0,312.0,117.0,30.0,6.0,150.0,147.0,150.0,303.0,120.0,114.0,123.0,5

In [57]:
del merged_final['Area_Code']
merged_final['Area_Description'] = merged_final['Area_Description'].replace(['Auckland Airport', 'Auckland-University'], 'Auckland')

In [58]:
numerical_columns = merged_final.select_dtypes(include=[np.number]).columns
merged_final[numerical_columns] = merged_final[numerical_columns].fillna(0)

In [59]:
merged_final = merged_final.groupby('Area_Description').sum().reset_index()

In [60]:
merged_final = merged_final.merge(pivot_crimes, left_on='Area_Description', right_on='Area Unit', how='outer')


In [61]:
merged_final = merged_final.drop_duplicates(subset='Area_Description', keep='first')

In [62]:
merged_final.head()

,Area_Description,2018.0,2018 Census,0-4 years,10-14 years,15-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,Median age(17),Under 15 years,15-29 years,30-64 years,Less than one year,1 year,2 years,3 years,4 years,5-9 years,10-19 years,No children,One child,Two children,Three children,Four children,Five children,No qualification,Level 1 certificate,Level 2 certificate,Level 3 certificate,Level 4 certificate,Level 5 diploma,Level 6 diploma,Bachelor degree and Level 7 qualification,Post-graduate and honours degrees,Masters degree,Doctorate degree,"$5,000 or less","$5,001 – $10,000","$10,001 – $20,000","$20,001 – $30,000","$30,001 – $50,000","$50,001 – $70,000",Median personal income($),Employed Full time,Employed Part time,Unemployed,Not in the Labour Force,Total stated,Managers,Professionals,Technicians and Trades Workers,Community and Personal Service Workers,Clerical and Administrative Workers,Sales Workers,Machinery Operators and Drivers,Manufacturing,Electricity Gas Water and Waste Services,Construction,Wholesale Trade,Retail Trade,Accommodation and Food Services,Transport Postal and Warehousing,Information Media and Telecommunications,Financial and Insurance Services,Rental Hiring and Real Estate Services,Professional Scientific and Technical Services,Administrative and Support Services,Public Administration and Safety,Education and Training,Health Care and Social Assistance,Arts and Recreation Services,Work at home,"Drive a private car, truck or van","Drive a company car, truck or van","Passenger in a car, truck, van or company bus",Train,Bicycle,Walk or jog,Area Unit,"Abduction, Harassment and Other Related Offences Against a Person",Acts Intended to Cause Injury,"Robbery, Extortion and Related Offences",Sexual Assault and Related Offences,Theft and Related Offences,"Unlawful Entry With Intent/Burglary, Break and Enter"
0,Akoranga,405.0,1167.0,42.0,27.0,153.0,105.0,84.0,57.0,45.0,33.0,48.0,39.0,36.0,39.0,33.0,48.0,69.0,87.0,43.9,108.0,342.0,291.0,21.0,18.0,18.0,18.0,12.0,48.0,126.0,237.0,66.0,111.0,87.0,36.0,24.0,147.0,87.0,78.0,195.0,51.0,36.0,75.0,132.0,45.0,36.0,6.0,168.0,78.0,201.0,267.0,192.0,81.0,22100.0,285.0,129.0,51.0,591.0,1056.0,48.0,96.0,45.0,72.0,42.0,60.0,9.0,21.0,3.0,33.0,18.0,45.0,51.0,15.0,9.0,12.0,3.0,51.0,24.0,15.0,27.0,48.0,18.0,12.0,135.0,21.0,9.0,0.0,3.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albany,2187.0,6741.0,315.0,375.0,498.0,636.0,441.0,462.0,471.0,426.0,474.0,411.0,351.0,309.0,273.0,258.0,228.0,171.0,106.0,1077.0,1572.0,2895.0,318.0,333.0,255.0,156.0,138.0,567.0,1164.0,939.0,423.0,852.0,360.0,99.0,48.0,588.0,426.0,381.0,666.0,321.0,231.0,252.0,1014.0,327.0,354.0,54.0,1143.0,303.0,816.0,726.0,1029.0,678.0,73800.0,2370.0,813.0,201.0,2286.0,5667.0,639.0,843.0,309.0,255.0,402.0,423.0,126.0,234.0,18.0,255.0,273.0,384.0,225.0,102.0,81.0,144.0,78.0,444.0,138.0,111.0,276.0,240.0,54.0,168.0,1521.0,381.0,105.0,3.0,15.0,93.0,Albany,1.0,80.0,15.0,4.0,734.0,63.0
2,Albany Heights,1053.0,3153.0,210.0,165.0,189.0,375.0,300.0,312.0,288.0,195.0,186.0,201.0,186.0,153.0,93.0,45.0,33.0,18.0,32.4,567.0,864.0,1521.0,132.0,177.0,114.0,120.0,102.0,393.0,576.0,486.0,234.0,312.0,117.0,30.0,6.0,150.0,147.0,150.0,303.0,120.0,114.0,123.0,585.0,183.0,174.0,15.0,507.0,132.0,291.0,225.0,480.0,363.0,36700.0,1371.0,390.0,111.0,714.0,2586.0,402.0,495.0,162.0,141.0,213.0,210.0,54.0,111.0,9.0,144.0,144.0,231.0,111.0,48.0,48.0,99.0,51.0,285.0,87.0,57.0,138.0,108.0,33.0,72.0,507.0,111.0,33.0,0.0,3.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alfriston,1059.0,3807.0,243.0,237.0,225.0,285.0,309.0,303.0,249.0,198.0,252.0,228.0,216.0,204.0,129.0,129.0,102.0,111.0,36.0,726.0,816.0,1656.0,45.0,54.0,60.0,51.0,48.0,294.0,699.0,408.0,219.0,384.0,240.0,105.0,54.0,561.0,261.0,207.0,297.0,204.0,144.0,138.0,492.0,147.0,99.0,15.0,432.0,144.0,522.0,381.0,582.0,495.0,32300.0,1572.0,336.0,93.0,1077.0,3078.0,363.0,435.0,201.0,138.0,252.0,234.0,14

In [63]:
df= dfinal

In [64]:
cols = ['Suburb'] + [col for col in df if col != 'Suburb']
df = df[cols]
df['Suburb'] = df['Suburb'].replace('Auckland Central', 'Auckland')


In [70]:
df.head()

,Suburb,Address,Price,Bedroom,Bathroom,Garage,Other Feature 1,Other Feature 2,Furniture,Type_Apartment,Type_Carpark,Type_House,Type_Studio,Type_Townhouse,Type_Unit,Type_Unknown,2018.0,2018 Census,0-4 years,10-14 years,15-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,Median age(17),Under 15 years,15-29 years,30-64 years,Less than one year,1 year,2 years,3 years,4 years,5-9 years,10-19 years,No children,One child,Two children,Three children,Four children,Five children,No qualification,Level 1 certificate,Level 2 certificate,Level 3 certificate,Level 4 certificate,Level 5 diploma,Level 6 diploma,Bachelor degree and Level 7 qualification,Post-graduate and honours degrees,Masters degree,Doctorate degree,"$5,000 or less","$5,001 – $10,000","$10,001 – $20,000","$20,001 – $30,000","$30,001 – $50,000","$50,001 – $70,000",Median personal income($),Employed Full time,Employed Part time,Unemployed,Not in the Labour Force,Total stated,Managers,Professionals,Technicians and Trades Workers,Community and Personal Service Workers,Clerical and Administrative Workers,Sales Workers,Machinery Operators and Drivers,Manufacturing,Electricity Gas Water and Waste Services,Construction,Wholesale Trade,Retail Trade,Accommodation and Food Services,Transport Postal and Warehousing,Information Media and Telecommunications,Financial and Insurance Services,Rental Hiring and Real Estate Services,Professional Scientific and Technical Services,Administrative and Support Services,Public Administration and Safety,Education and Training,Health Care and Social Assistance,Arts and Recreation Services,Work at home,"Drive a private car, truck or van","Drive a company car, truck or van","Passenger in a car, truck, van or company bus",Train,Bicycle,Walk or jog,Area Unit,"Abduction, Harassment and Other Related Offences Against a Person",Acts Intended to Cause Injury,"Robbery, Extortion and Related Offences",Sexual Assault and Related Offences,Theft and Related Offences,"Unlawful Entry With Intent/Burglary, Break and Enter"
86,Auckland,"9/3 Eden Crescent, Auckland Central, Auckland ...",620,2,1,1,NaN,NaN,0,1,0,0,0,0,0,0,213.0,735.0,66.0,33.0,39.0,69.0,75.0,69.0,60.0,36.0,45.0,60.0,57.0,39.0,24.0,9.0,12.0,3.0,64.8,141.0,180.0,366.0,21.0,24.0,9.0,3.0,15.0,39.0,33.0,81.0,30.0,36.0,15.0,21.0,3.0,69.0,42.0,45.0,60.0,54.0,18.0,21.0,72.0,27.0,18.0,3.0,78.0,30.0,87.0,69.0,150.0,96.0,66800.0,351.0,69.0,21.0,153.0,594.0,57.0,90.0,45.0,42.0,51.0,33.0,42.0,45.0,3.0,33.0,18.0,30.0,42.0,48.0,9.0,6.0,9.0,36.0,21.0,21.0,33.0,21.0,12.0,30.0,120.0,39.0,9.0,0.0,3.0,36.0,Auckland,1.0,262.0,42.0,11.0,1628.0,219.0
222,Auckland,"2B/2 White Street, Auckland Central, Auckland ...",Auckland Central insights,2,1,<NA>,None,None,1,1,0,0,0,0,0,0,213.0,735.0,66.0,33.0,39.0,69.0,75.0,69.0,60.0,36.0,45.0,60.0,57.0,39.0,24.0,9.0,12.0,3.0,64.8,141.0,180.0,366.0,21.0,24.0,9.0,3.0,15.0,39.0,33.0,81.0,30.0,36.0,15.0,21.0,3.0,69.0,42.0,45.0,60.0,54.0,18.0,21.0,72.0,27.0,18.0,3.0,78.0,30.0,87.0,69.0,150.0,96.0,66800.0,351.0,69.0,21.0,153.0,594.0,57.0,90.0,45.0,42.0,51.0,33.0,42.0,45.0,3.0,33.0,18.0,30.0,42.0,48.0,9.0,6.0,9.0,36.0,21.0,21.0,33.0,21.0,12.0,30.0,120.0,39.0,9.0,0.0,3.0,36.0,Auckland,1.0,262.0,42.0,11.0,1628.0,219.0
109,Auckland,"406/138 Anzac Ave, Auckland Central, Auckland ...",320,1,1,<NA>,1 ensuite,NaN,0,1,0,0,0,0,0,0,213.0,735.0,66.0,33.0,39.0,69.0,75.0,69.0,60.0,36.0,45.0,60.0,57.0,39.0,24.0,9.0,12.0,3.0,64.8,141.0,180.0,366.0,21.0,24.0,9.0,3.0,15.0,39.0,33.0,81.0,30.0,36.0,15.0,21.0,3.0,69.0,42.0,45.0,60.0,54.0,18.0,21.0,72.0,27.0,18.0,3.0,78.0,30.0,87.0,69.0,150.0,96.0,66800.0,351.0,69.0,21.0,153.0,594.0,57.0,90.0,45.0,42.0,51.0,33.0,42.0,45.0,3.0,33.0,18.0,30.0,42.0,48.0,9.0,6.0,9.0,36.0,21.0,21.0,33.0,21.0,12.0,30.0,120.0,39.0,9.0,0.0,3.0,36.0,Auckland,1.0,262.0,42.0,11.0,1628.0,219.0
110,Auckland,"121/72 Nelson St , Auckland Central, Auckland ...",470,2,1,<NA>,NaN,NaN,0,

In [67]:
df = df.merge(merged_final, left_on='Suburb', right_on='Area_Description', how='left')

In [68]:
df.drop('Area_Description', axis=1, inplace=True)
df = df.sort_values(by='Suburb')

In [69]:
df.head()

,Suburb,Address,Price,Bedroom,Bathroom,Garage,Other Feature 1,Other Feature 2,Furniture,Type_Apartment,Type_Carpark,Type_House,Type_Studio,Type_Townhouse,Type_Unit,Type_Unknown,2018.0,2018 Census,0-4 years,10-14 years,15-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,Median age(17),Under 15 years,15-29 years,30-64 years,Less than one year,1 year,2 years,3 years,4 years,5-9 years,10-19 years,No children,One child,Two children,Three children,Four children,Five children,No qualification,Level 1 certificate,Level 2 certificate,Level 3 certificate,Level 4 certificate,Level 5 diploma,Level 6 diploma,Bachelor degree and Level 7 qualification,Post-graduate and honours degrees,Masters degree,Doctorate degree,"$5,000 or less","$5,001 – $10,000","$10,001 – $20,000","$20,001 – $30,000","$30,001 – $50,000","$50,001 – $70,000",Median personal income($),Employed Full time,Employed Part time,Unemployed,Not in the Labour Force,Total stated,Managers,Professionals,Technicians and Trades Workers,Community and Personal Service Workers,Clerical and Administrative Workers,Sales Workers,Machinery Operators and Drivers,Manufacturing,Electricity Gas Water and Waste Services,Construction,Wholesale Trade,Retail Trade,Accommodation and Food Services,Transport Postal and Warehousing,Information Media and Telecommunications,Financial and Insurance Services,Rental Hiring and Real Estate Services,Professional Scientific and Technical Services,Administrative and Support Services,Public Administration and Safety,Education and Training,Health Care and Social Assistance,Arts and Recreation Services,Work at home,"Drive a private car, truck or van","Drive a company car, truck or van","Passenger in a car, truck, van or company bus",Train,Bicycle,Walk or jog,Area Unit,"Abduction, Harassment and Other Related Offences Against a Person",Acts Intended to Cause Injury,"Robbery, Extortion and Related Offences",Sexual Assault and Related Offences,Theft and Related Offences,"Unlawful Entry With Intent/Burglary, Break and Enter"
86,Auckland,"9/3 Eden Crescent, Auckland Central, Auckland ...",620,2,1,1,NaN,NaN,0,1,0,0,0,0,0,0,213.0,735.0,66.0,33.0,39.0,69.0,75.0,69.0,60.0,36.0,45.0,60.0,57.0,39.0,24.0,9.0,12.0,3.0,64.8,141.0,180.0,366.0,21.0,24.0,9.0,3.0,15.0,39.0,33.0,81.0,30.0,36.0,15.0,21.0,3.0,69.0,42.0,45.0,60.0,54.0,18.0,21.0,72.0,27.0,18.0,3.0,78.0,30.0,87.0,69.0,150.0,96.0,66800.0,351.0,69.0,21.0,153.0,594.0,57.0,90.0,45.0,42.0,51.0,33.0,42.0,45.0,3.0,33.0,18.0,30.0,42.0,48.0,9.0,6.0,9.0,36.0,21.0,21.0,33.0,21.0,12.0,30.0,120.0,39.0,9.0,0.0,3.0,36.0,Auckland,1.0,262.0,42.0,11.0,1628.0,219.0
222,Auckland,"2B/2 White Street, Auckland Central, Auckland ...",Auckland Central insights,2,1,<NA>,None,None,1,1,0,0,0,0,0,0,213.0,735.0,66.0,33.0,39.0,69.0,75.0,69.0,60.0,36.0,45.0,60.0,57.0,39.0,24.0,9.0,12.0,3.0,64.8,141.0,180.0,366.0,21.0,24.0,9.0,3.0,15.0,39.0,33.0,81.0,30.0,36.0,15.0,21.0,3.0,69.0,42.0,45.0,60.0,54.0,18.0,21.0,72.0,27.0,18.0,3.0,78.0,30.0,87.0,69.0,150.0,96.0,66800.0,351.0,69.0,21.0,153.0,594.0,57.0,90.0,45.0,42.0,51.0,33.0,42.0,45.0,3.0,33.0,18.0,30.0,42.0,48.0,9.0,6.0,9.0,36.0,21.0,21.0,33.0,21.0,12.0,30.0,120.0,39.0,9.0,0.0,3.0,36.0,Auckland,1.0,262.0,42.0,11.0,1628.0,219.0
109,Auckland,"406/138 Anzac Ave, Auckland Central, Auckland ...",320,1,1,<NA>,1 ensuite,NaN,0,1,0,0,0,0,0,0,213.0,735.0,66.0,33.0,39.0,69.0,75.0,69.0,60.0,36.0,45.0,60.0,57.0,39.0,24.0,9.0,12.0,3.0,64.8,141.0,180.0,366.0,21.0,24.0,9.0,3.0,15.0,39.0,33.0,81.0,30.0,36.0,15.0,21.0,3.0,69.0,42.0,45.0,60.0,54.0,18.0,21.0,72.0,27.0,18.0,3.0,78.0,30.0,87.0,69.0,150.0,96.0,66800.0,351.0,69.0,21.0,153.0,594.0,57.0,90.0,45.0,42.0,51.0,33.0,42.0,45.0,3.0,33.0,18.0,30.0,42.0,48.0,9.0,6.0,9.0,36.0,21.0,21.0,33.0,21.0,12.0,30.0,120.0,39.0,9.0,0.0,3.0,36.0,Auckland,1.0,262.0,42.0,11.0,1628.0,219.0
110,Auckland,"121/72 Nelson St , Auckland Central, Auckland ...",470,2,1,<NA>,NaN,NaN,0,

In [ ]:
df.to_csv('prediction.csv', index=False)